<a href="https://colab.research.google.com/github/codesanya/dobbe-xray-preprocessing/blob/main/notebooks/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os
data_path = '/content/drive/MyDrive/Images_Data_science_intern'
print(os.listdir(data_path))  # Should list your .dcm, .rvg files


['IS20250116_180218_7445_56958340.dcm', 'R9.rvg', 'R5.rvg', 'IS20250115_190348_9148_86297118.dcm', 'R6.rvg', 'R10.rvg', 'R7.rvg', 'IS20250115_171841_9465_61003253.dcm', 'IS20250218_193552_3393_78829426.dcm', 'R4.rvg', 'IS20250115_191316_7227_10120577.dcm', 'IS20250221_192657_5718_56712676.dcm', 'Reference_Output_Quality.jpg', 'IS20250218_193621_8940_10081171.dcm']


In [3]:
# Clone your GitHub repo into Colab
!git clone https://github.com/codesanya/dobbe-xray-preprocessing.git
%cd dobbe-xray-preprocessing


Cloning into 'dobbe-xray-preprocessing'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 20 (delta 2), reused 13 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (20/20), done.
Resolving deltas: 100% (2/2), done.
/content/dobbe-xray-preprocessing
